In [ ]:
# --- 1. IMPORTAÇÕES ---
import os
import panel as pn
import pandas as pd
import psycopg2 as pg
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Date, DateTime, Text, Numeric, CheckConstraint, or_, Time
from sqlalchemy.orm import relationship, declarative_base, sessionmaker
try: from dotenv import load_dotenv; load_dotenv()
except: pass

pn.extension('tabulator', notifications=True, design='material')
pn.config.sizing_mode = 'stretch_width'


In [ ]:
# --- 2. ENV ---
DB_HOST = os.getenv('DB_HOST', 'localhost')
DB_PORT = os.getenv('DB_PORT', '5432')
DB_NAME = os.getenv('DB_NAME', 'AgendaCultural')
DB_USER = os.getenv('DB_USER', 'postgres')
DB_PASS = os.getenv('DB_PASS', 'postgres')
def sqlalchemy_url(): return f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'


In [ ]:
# --- 3. CONEXÃO ---
_engine = create_engine(sqlalchemy_url(), pool_pre_ping=True)
Session = sessionmaker(bind=_engine)
Base = declarative_base()
cnx = _engine
flag = ''


In [ ]:
# --- 4. MODELOS ---
from sqlalchemy.sql import func
class Role(Base):
    __tablename__ = 'role'
    id_papel = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String(50), nullable=False, unique=True)
    descricao = Column(Text)

class Usuario(Base):
    __tablename__ = 'usuario'
    id_usuario = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String(100), nullable=False)
    email = Column(String(255), nullable=False, unique=True)
    cpf_rg = Column(String(20), nullable=False, unique=True)
    senha_hash = Column(String(255), nullable=False)
    id_papel = Column(Integer, ForeignKey('role.id_papel'), nullable=False)
    role_rel = relationship('Role')
    avaliacoes = relationship('AvaliacaoEvento', back_populates='usuario')

class EspacoCultural(Base):
    __tablename__ = 'espaco_cultural'
    id_espaco_cult = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String(100), nullable=False)
    rua = Column(String(100), nullable=False)
    numero = Column(String(10), nullable=False)
    bairro = Column(String(60), nullable=False)
    eventos = relationship('Evento', back_populates='espaco')

class Evento(Base):
    __tablename__ = 'evento'
    id_evento = Column(Integer, primary_key=True, autoincrement=True)
    titulo = Column(String(150), nullable=False)
    descricao = Column(Text)
    categoria = Column(String(60))
    capacidade = Column(Integer)
    data_inicio = Column(DateTime, nullable=False)
    data_fim = Column(DateTime, nullable=False)
    preco = Column(Numeric(10,2), default=0)
    status = Column(String(20), nullable=False)
    id_espaco_cult = Column(Integer, ForeignKey('espaco_cultural.id_espaco_cult'), nullable=False)
    espaco = relationship('EspacoCultural', back_populates='eventos')
    avaliacoes = relationship('AvaliacaoEvento', back_populates='evento')

class AvaliacaoEvento(Base):
    __tablename__ = 'usuario_avalia_evento'
    id_avaliacao = Column(Integer, primary_key=True, autoincrement=True)
    nota = Column(Integer, nullable=False)
    comentario = Column(Text)
    data = Column(DateTime, server_default=func.now(), onupdate=func.now())
    id_usuario = Column(Integer, ForeignKey('usuario.id_usuario'), nullable=False)
    id_evento = Column(Integer, ForeignKey('evento.id_evento'), nullable=False)
    usuario = relationship('Usuario', back_populates='avaliacoes')
    evento = relationship('Evento', back_populates='avaliacoes')

class Artista(Base):
    __tablename__ = 'artista'
    id_artista = Column(Integer, primary_key=True, autoincrement=True)
    nome = Column(String(100), nullable=False)
    cpf_rg = Column(String(20), nullable=False, unique=True)
    email = Column(String(255), nullable=False, unique=True)
    telefone = Column(String(20), nullable=False, unique=True)
    descricao = Column(Text)

class ArtistaApresentaEvento(Base):
    __tablename__ = 'artista_apresenta_evento'
    id_evento = Column(Integer, ForeignKey('evento.id_evento'), primary_key=True)
    id_artista = Column(Integer, ForeignKey('artista.id_artista'), primary_key=True)
    data = Column(DateTime)
    hora = Column(Time)

Base.metadata.create_all(_engine)


In [ ]:
# --- 5. AUTH ---
CURRENT_USER = None  # None = Visitante

def verificar_senha(inp, db): return str(inp) == str(db)

def tentar_login(email, senha):
    global CURRENT_USER
    s = Session()
    try:
        u = s.query(Usuario).filter_by(email=email).first()
        if u and verificar_senha(senha, u.senha_hash):
            r = u.role_rel.nome if u.role_rel else 'Comum'
            CURRENT_USER = {'id': u.id_usuario, 'nome': u.nome, 'role': r, 'email': u.email}
            return True
    finally: s.close()
    return False

def logout():
    global CURRENT_USER
    CURRENT_USER = None

# --- GLOBAL HELPER: get_role ---
def get_role():
    if not CURRENT_USER:
        return 'VISITANTE'
    # Return DB role directly (assumed UPPERCASE) or force upper
    return str(CURRENT_USER.get('role', 'VISITANTE')).upper()


In [ ]:
# --- 6. PERMISSÕES E UI DINÂMICA ---

def atualizar_interface():
    role = get_role()
    
    # --- Header ---
    if role == 'VISITANTE':
        header_user.object = 'Modo Visitante'
        btnHeaderLogin.visible = True;
        btnHeaderLogout.visible = False
    else:
        header_user.object = f"Usuário: {CURRENT_USER['nome']} ({get_role()})"
        btnHeaderLogin.visible = False;
        btnHeaderLogout.visible = True
    
    # --- CRUD VISIBILITY ---
    can_edit_content = (role in ['ADMIN', 'GERENTE'])
    row_crud_ev_btns.visible = can_edit_content
    row_crud_esp_btns.visible = can_edit_content
    col_aval_ev.visible = (role == 'COMUM')

    # --- Aba Usuários e Artistas ---
    can_manage_users = (role in ['ADMIN', 'GERENTE'])
    # Updated list to include Artistas
    tabs_main.objects = [tab_eventos, tab_espacos] if not can_manage_users else [tab_eventos, tab_espacos, tab_usuarios, tab_artistas]
    
    # OPTIONS UPDATE: UPPERCASE
    if role == 'ADMIN':
        user_role_select.options = ['GERENTE', 'COMUM']
    elif role == 'GERENTE':
        user_role_select.options = ['COMUM']

    # REFRESH VIEWS
    try: on_con_ev()
    except: pass
    try: on_con_esp()
    except: pass
    try: on_con_users()
    except: pass
    try: on_con_art()
    except: pass


In [ ]:
# --- 7. EVENTOS (Full Consolidated) ---
import functools
from sqlalchemy import text

# --- WIDGETS ---
ev_id = pn.widgets.IntInput(name='ID', disabled=True, width=80)
ev_titulo = pn.widgets.TextInput(name='Título')
ev_cat = pn.widgets.TextInput(name='Categoria')
ev_status = pn.widgets.Select(name='Status', options=['', 'Agendado', 'Realizado', 'Cancelado'], value='')
ev_dt_ini = pn.widgets.DatePicker(name='Data Início')
ev_dt_fim = pn.widgets.DatePicker(name='Data Fim')

# Extra Fields
ev_capacidade = pn.widgets.IntInput(name='Capacidade', start=0)
ev_preco = pn.widgets.FloatInput(name='Preço (R$)', start=0.0, step=0.01)
ev_descricao = pn.widgets.TextAreaInput(name='Descrição', height=100)
ev_artistas = pn.widgets.MultiChoice(name='Artistas', option_limit=100)

# Buttons
btnConEv = pn.widgets.Button(name='Consultar', button_type='primary')
btnLimparEv = pn.widgets.Button(name='Limpar', button_type='default')
btnInsEv = pn.widgets.Button(name='Inserir', button_type='success')
btnAttEv = pn.widgets.Button(name='Atualizar', button_type='warning')
btnDelEv = pn.widgets.Button(name='Excluir', button_type='danger')

# Layout Form
col_inputs_ev = pn.Column(
    '### Gestão de Eventos',
    pn.Row(ev_id, ev_status),
    ev_titulo,
    pn.Row(ev_cat, ev_capacidade, ev_preco),
    pn.Row(ev_dt_ini, ev_dt_fim, ev_artistas),
    ev_descricao,
    pn.Row(btnConEv, btnLimparEv)
)
row_crud_ev_btns = pn.Row(btnInsEv, btnAttEv, btnDelEv)
col_ev_form = pn.Column(col_inputs_ev, row_crud_ev_btns)

# Avaliação Widget
aval_nota = pn.widgets.IntSlider(name='Nota', start=1, end=5, step=1, value=5)
aval_coment = pn.widgets.TextAreaInput(name='Comentário', height=100)
btnEnviarAval = pn.widgets.Button(name='Avaliar', button_type='primary')
btnDenunciar = pn.widgets.Button(name='Denunciar', button_type='danger')
col_aval_ev = pn.Column('### Avaliar Evento', aval_nota, aval_coment, pn.Row(btnEnviarAval, btnDenunciar))

# --- VIEWS ---

# 1. Admin Table
tab_ev_admin = pn.widgets.Tabulator(pagination='remote', page_size=15, layout='fit_columns', 
                                    selectable=1, show_index=False, disabled=True, 
                                    sizing_mode='stretch_width', theme='bootstrap4')

# 2. Public Feed
btnEvAnt = pn.widgets.Button(name='< Anterior', width=100)
btnEvProx = pn.widgets.Button(name='Próximo >', width=100)
lblEvPage = pn.widgets.StaticText(value='Página 1', align='center')
feed_ev = pn.Column(scroll=True, height=800, sizing_mode='stretch_width')
col_feed_nav = pn.Column(pn.Row(btnEvAnt, lblEvPage, btnEvProx), feed_ev)

ev_page = 0
EV_PAGE_SIZE = 2

# --- LOGIC ---

def atualizar_opcoes_artistas():
    try:
        df = pd.read_sql_query('SELECT id_artista, nome FROM artista ORDER BY nome', cnx)
        opts = {row['nome']: str(row['id_artista']) for i, row in df.iterrows()}
        ev_artistas.options = opts
    except: pass

def carregar_artistas_evento(eid):
    try:
        atualizar_opcoes_artistas()
        q = f'SELECT id_artista FROM artista_apresenta_evento WHERE id_evento={eid}'
        df = pd.read_sql_query(q, cnx)
        ids = [str(x) for x in df['id_artista'].tolist()]
        ev_artistas.value = ids
    except: pass

def carregar_form(row):
    try:
        ev_id.value = int(row['id_evento'])
        ev_titulo.value = str(row['titulo'])
        ev_cat.value = str(row['categoria']) if row['categoria'] else ''
        ev_status.value = str(row['status']) if row['status'] else ''
        ev_capacidade.value = int(row['capacidade']) if row['capacidade'] else 0
        ev_preco.value = float(row['preco']) if row['preco'] else 0.0
        ev_descricao.value = str(row['descricao']) if row['descricao'] else ''
        ev_dt_ini.value = pd.to_datetime(row['data_inicio']).date() if row['data_inicio'] else None
        ev_dt_fim.value = pd.to_datetime(row['data_fim']).date() if row['data_fim'] else None
        carregar_artistas_evento(ev_id.value)
        pn.state.notifications.info(f"Carregado: {row['titulo']}")
    except Exception as e: pn.state.notifications.error(f'Erro load: {e}')

def on_sel_admin(event):
    if not event.new: return
    try: carregar_form(tab_ev_admin.value.iloc[event.new[0]])
    except: pass
tab_ev_admin.param.watch(on_sel_admin, 'selection')

def criar_widget_publico(row, can_eval):
    dt_str = f"{row['data_inicio'].strftime('%d/%m/%Y')} a {row['data_fim'].strftime('%d/%m/%Y')}" if row['data_inicio'] else "N/D"
    data = {
        'Campo': ['Título', 'Categoria', 'Status', 'Data', 'Local', 'Avaliação', 'Artistas', 'Preço', 'Descrição'],
        'Valor': [
            str(row['titulo']), str(row['categoria']), str(row['status']), dt_str,
            str(row['local']), f"⭐ {row['avaliacao']}", str(row['artistas'] or 'Nenhum'),
            f"R$ {row['preco']}", str(row['descricao'] or '')
        ]
    }
    df_mini = pd.DataFrame(data)
    # Selectable logic for Public
    tbl = pn.widgets.Tabulator(df_mini, show_index=False, disabled=False, 
                               configuration={'headerVisible': False}, 
                               editors={'Campo': None, 'Valor': None}, selectable=1,
                               sizing_mode='stretch_width', theme='bootstrap4')
    
    def on_sel_pub(event):
        if event.new:
            try:
                carregar_form(row)
                try: tbl.selection = []
                except: pass
            except: pass
    tbl.param.watch(on_sel_pub, 'selection')

    # Aval Button
    btns = []
    if can_eval:
        b = pn.widgets.Button(name='⭐ Avaliar', button_type='primary', width=100)
        b.on_click(lambda e: setattr(ev_id, 'value', int(row['id_evento'])))
        btns.append(b)
    return pn.Column(tbl, pn.Row(*btns), pn.layout.Divider(), margin=(10, 10))

def fetch_events(limit=None, offset=0):
    t = ev_titulo.value.replace("'", "''")
    c = ev_cat.value.replace("'", "''")
    st = ev_status.value
    
    query = f"""
        SELECT 
            e.id_evento, e.titulo, e.descricao, e.categoria, e.capacidade,
            e.data_inicio, e.data_fim, e.preco, e.status,
            ec.nome as "local",
            ROUND(COALESCE(AVG(ae.nota), 0), 1) as avaliacao,
            STRING_AGG(DISTINCT art.nome, ', ') as artistas
        FROM evento e
        JOIN espaco_cultural ec ON e.id_espaco_cult = ec.id_espaco_cult
        LEFT JOIN usuario_avalia_evento ae ON e.id_evento = ae.id_evento
        LEFT JOIN artista_apresenta_evento aae ON e.id_evento = aae.id_evento
        LEFT JOIN artista art ON aae.id_artista = art.id_artista
        WHERE 
            ('{t}'='' OR e.titulo ILIKE '%%{t}%%') 
            AND ('{c}'='' OR e.categoria ILIKE '%%{c}%%')
            AND ('{st}'='' OR e.status = '{st}')
        GROUP BY e.id_evento, ec.nome
        ORDER BY e.data_inicio DESC
    """
    if limit:
        query += f" LIMIT {limit} OFFSET {offset} "
    return pd.read_sql_query(query, cnx)

def on_con_ev(e=None):
    role = get_role()
    is_admin_mode = (role in ['ADMIN', 'GERENTE'])
    
    tab_ev_admin.visible = is_admin_mode
    col_feed_nav.visible = not is_admin_mode
    
    # Input Visibility
    ev_descricao.visible = is_admin_mode
    ev_capacidade.visible = is_admin_mode
    ev_preco.visible = is_admin_mode
    ev_artistas.visible = is_admin_mode

    actual_fetch(is_admin_mode)

def actual_fetch(is_admin):
    if is_admin:
        try: 
             tab_ev_admin.value = fetch_events()
             atualizar_opcoes_artistas()
        except Exception as x: pn.state.notifications.error(f'{x}')
    else:
        atualizar_feed_publico()

def atualizar_feed_publico():
    feed_ev.clear()
    feed_ev.append(pn.indicators.LoadingSpinner(value=True, width=30))
    try:
        df = fetch_events(limit=EV_PAGE_SIZE, offset=ev_page * EV_PAGE_SIZE)
        feed_ev.clear()
        if df.empty:
             feed_ev.append(pn.pane.Alert('Fim da lista.', alert_type='warning'))
             return
        
        can_eval = (get_role() == 'COMUM')
        for idx, row in df.iterrows():
            feed_ev.append(criar_widget_publico(row, can_eval))
    except Exception as x: pn.state.notifications.error(f'{x}')

def nav_ant(e):
    global ev_page
    if ev_page > 0:
        ev_page -= 1
        lblEvPage.value = f'Página {ev_page + 1}'
        atualizar_feed_publico()

def nav_prox(e):
    global ev_page
    ev_page += 1
    lblEvPage.value = f'Página {ev_page + 1}'
    atualizar_feed_publico()

btnEvAnt.on_click(nav_ant); btnEvProx.on_click(nav_prox)

# --- CRUD LOGIC ---
def on_ins_ev(e=None):
    s = Session()
    try:
        if s.query(Evento).filter_by(titulo=ev_titulo.value).first():
             pn.state.notifications.error('Título duplicado!')
             return
        if not ev_dt_ini.value or not ev_dt_fim.value: 
            raise Exception('Datas obrigatórias')
        
        new_ev = Evento(
            titulo=ev_titulo.value, categoria=ev_cat.value, 
            status=ev_status.value or 'Agendado', 
            data_inicio=ev_dt_ini.value, data_fim=ev_dt_fim.value, 
            id_espaco_cult=1, capacidade=ev_capacidade.value,
            preco=ev_preco.value, descricao=ev_descricao.value)
        s.add(new_ev)
        s.flush() # get ID
        # Insert Artists (Updated Schema: id_evento, id_artista, data, hora)
        dt_val = f"'{ev_dt_ini.value}'" if ev_dt_ini.value else 'NULL'
        for art_id_str in ev_artistas.value:
            s.execute(text(f'INSERT INTO artista_apresenta_evento (id_artista, id_evento, data, hora) VALUES ({art_id_str}, {new_ev.id_evento}, {dt_val}, NULL)'))
        s.commit(); pn.state.notifications.success('Inserido!'); on_con_ev()
    except Exception as x: pn.state.notifications.error(f'{x}')
    finally: s.close()

def on_att_ev(e=None):
    if not ev_id.value: return
    s = Session()
    try:
        o = s.get(Evento, ev_id.value)
        if o: 
             # Minimal update logic for demo
             o.titulo=ev_titulo.value; o.categoria=ev_cat.value
             if ev_status.value: o.status=ev_status.value
             if ev_dt_ini.value: o.data_inicio=ev_dt_ini.value
             if ev_dt_fim.value: o.data_fim=ev_dt_fim.value
             o.capacidade=ev_capacidade.value; o.preco=ev_preco.value
             o.descricao=ev_descricao.value
             # Update Artists: Delete All, Re-insert
             dt_val = f"'{ev_dt_ini.value}'" if ev_dt_ini.value else 'NULL'
             s.execute(text(f'DELETE FROM artista_apresenta_evento WHERE id_evento={o.id_evento}'))
             for art_id_str in ev_artistas.value:
                 s.execute(text(f'INSERT INTO artista_apresenta_evento (id_artista, id_evento, data, hora) VALUES ({art_id_str}, {o.id_evento}, {dt_val}, NULL)'))
             s.commit(); pn.state.notifications.success('Atualizado!'); on_con_ev()
    finally: s.close()

def on_del_ev(e=None):
    s = Session()
    try: 
        if o:=s.get(Evento, ev_id.value): s.delete(o); s.commit(); pn.state.notifications.success('Excluído!'); on_con_ev()
    finally: s.close()

def on_limpar_ev(e=None):
    ev_id.value = 0; ev_titulo.value = ''; ev_cat.value = ''; ev_status.value = ''
    ev_dt_ini.value = None; ev_dt_fim.value = None
    ev_capacidade.value = 0; ev_preco.value = 0.0; ev_descricao.value = ''
    ev_artistas.value = []
    tab_ev_admin.selection = []
    on_con_ev()

def on_aval_ev(e=None):
    if not CURRENT_USER: return
    if not ev_id.value: 
        pn.state.notifications.warning('Selecione!')
        return
    s = Session()
    try:
        av = s.query(AvaliacaoEvento).filter_by(id_evento=ev_id.value, id_usuario=CURRENT_USER['id']).first()
        if av:
            av.nota = aval_nota.value; av.comentario = aval_coment.value
            pn.state.notifications.success('Nota atualizada!')
        else:
            s.add(AvaliacaoEvento(id_evento=ev_id.value, id_usuario=CURRENT_USER['id'], nota=aval_nota.value, comentario=aval_coment.value))
            pn.state.notifications.success('Avaliado!')
        s.commit(); on_con_ev()
    except Exception as x: pn.state.notifications.error(f'{x}')
    finally: s.close()

btnConEv.on_click(on_con_ev); btnLimparEv.on_click(on_limpar_ev)
btnInsEv.on_click(on_ins_ev); btnAttEv.on_click(on_att_ev); btnDelEv.on_click(on_del_ev)
btnEnviarAval.on_click(on_aval_ev); btnDenunciar.on_click(lambda e: pn.state.notifications.success('Denúncia simulada!'))

on_con_ev()
tab_eventos = pn.Row(pn.Column(col_ev_form, col_aval_ev), tab_ev_admin, col_feed_nav, name='Eventos')


In [ ]:
# --- 8. ESPAÇOS (Hybrid View - NO EVAL) ---

# Widgets CRUD
esp_id = pn.widgets.IntInput(name='ID', disabled=True, width=80)
esp_nome = pn.widgets.TextInput(name='Nome')
esp_rua = pn.widgets.TextInput(name='Rua')
esp_num = pn.widgets.TextInput(name='Número')
esp_bairro = pn.widgets.TextInput(name='Bairro')

btnConEsp = pn.widgets.Button(name='Consultar', button_type='primary')
btnLimparEsp = pn.widgets.Button(name='Limpar', button_type='default')
btnInsEsp = pn.widgets.Button(name='Inserir', button_type='success')
btnAttEsp = pn.widgets.Button(name='Atualizar', button_type='warning')
btnDelEsp = pn.widgets.Button(name='Excluir', button_type='danger')

# Layouts
col_inputs_esp = pn.Column('### Gestão de Espaços', 
                           esp_id, esp_nome, esp_rua, esp_num, esp_bairro,
                           pn.Row(btnConEsp, btnLimparEsp))

row_crud_esp_btns = pn.Row(btnInsEsp, btnAttEsp, btnDelEsp)
col_esp_main = pn.Column(col_inputs_esp, row_crud_esp_btns)

# --- VIEW 1: ADMIN ---
tab_esp_admin = pn.widgets.Tabulator(pagination='remote', page_size=10, layout='fit_columns', 
                                     selectable=1, show_index=False, disabled=True,
                                     sizing_mode='stretch_width', theme='bootstrap4')

# --- VIEW 2: PUBLIC ---
btnEspAnt = pn.widgets.Button(name='< Anterior', width=100)
btnEspProx = pn.widgets.Button(name='Próximo >', width=100)
lblEspPage = pn.widgets.StaticText(value='Página 1', align='center')
feed_esp = pn.Column(scroll=True, height=800, sizing_mode='stretch_width')
col_feed_esp_nav = pn.Column(pn.Row(btnEspAnt, lblEspPage, btnEspProx), feed_esp)

esp_page = 0
ESP_PAGE_SIZE = 4

# --- LOGIC ---
def carregar_form_esp(row):
    try:
        esp_id.value = int(row['id_espaco_cult'])
        esp_nome.value = str(row['nome'])
        esp_rua.value = str(row['rua']) if row['rua'] else ''
        esp_num.value = str(row['numero']) if row['numero'] else ''
        esp_bairro.value = str(row['bairro']) if row['bairro'] else ''
    except: pass

def on_sel_esp_admin(event):
    if not event.new: return
    try:
        row = tab_esp_admin.value.iloc[event.new[0]]
        carregar_form_esp(row)
    except: pass
tab_esp_admin.param.watch(on_sel_esp_admin, 'selection')

def criar_widget_espaco_publico(row, can_eval=False):
    data = {
        'Campo': ['Nome', 'Rua', 'Número', 'Bairro'],
        'Valor': [
             str(row['nome']), str(row['rua']), str(row['numero']), str(row['bairro'])
        ]
    }
    df_mini = pd.DataFrame(data)
    # Updated: Enable Selection
    tbl = pn.widgets.Tabulator(df_mini, show_index=False, disabled=False, 
                               configuration={'headerVisible': False}, 
                               editors={'Campo': None, 'Valor': None}, selectable=1,
                               sizing_mode='stretch_width', theme='bootstrap4')
    
    def on_sel_pub(event):
        if event.new:
            try:
                carregar_form_esp(row)
                try: tbl.selection = []
                except: pass
            except: pass
    tbl.param.watch(on_sel_pub, 'selection')
    
    return pn.Column(tbl, pn.layout.Divider(), margin=(10, 10))
def fetch_espacos(limit=None, offset=0):
    n = esp_nome.value.replace("'", "''")
    r = esp_rua.value.replace("'", "''")
    b = esp_bairro.value.replace("'", "''")
    
    query = f"""
        SELECT * FROM espaco_cultural 
        WHERE 
            ('{n}'='' OR nome ILIKE '%%{n}%%') 
            AND ('{r}'='' OR rua ILIKE '%%{r}%%')
            AND ('{b}'='' OR bairro ILIKE '%%{b}%%')
        ORDER BY id_espaco_cult
    """
    if limit:
        query += f" LIMIT {limit} OFFSET {offset} "
    return pd.read_sql_query(query, cnx)

def on_con_esp(e=None):
    role = get_role()
    is_admin_mode = (role in ['ADMIN', 'GERENTE'])
    
    tab_esp_admin.visible = is_admin_mode
    col_feed_esp_nav.visible = not is_admin_mode
    row_crud_esp_btns.visible = is_admin_mode
    
    if is_admin_mode:
        try: tab_esp_admin.value = fetch_espacos()
        except: pass
    else:
        atualizar_feed_esp_publico()

def atualizar_feed_esp_publico():
    feed_esp.clear()
    feed_esp.append(pn.indicators.LoadingSpinner(value=True, width=30))
    try:
        df = fetch_espacos(limit=ESP_PAGE_SIZE, offset=esp_page * ESP_PAGE_SIZE)
        feed_esp.clear()
        if df.empty:
             feed_esp.append(pn.pane.Alert('Fim da lista.', alert_type='warning'))
             return
        
        for idx, row in df.iterrows():
            feed_esp.append(criar_widget_espaco_publico(row))
    except Exception as x: pn.state.notifications.error(f'{x}')

def nav_esp_ant(e):
    global esp_page
    if esp_page > 0:
        esp_page -= 1
        lblEspPage.value = f'Página {esp_page + 1}'
        atualizar_feed_esp_publico()

def nav_esp_prox(e):
    global esp_page
    esp_page += 1
    lblEspPage.value = f'Página {esp_page + 1}'
    atualizar_feed_esp_publico()

btnEspAnt.on_click(nav_esp_ant); btnEspProx.on_click(nav_esp_prox)

def on_limpar_esp(e=None):
    esp_id.value = 0; esp_nome.value = ''; esp_rua.value = ''; esp_num.value = ''; esp_bairro.value = ''
    tab_esp_admin.selection = []
    on_con_esp()

def on_ins_esp(e=None):
    s = Session()
    try:
        if s.query(EspacoCultural).filter_by(nome=esp_nome.value).first():
            pn.state.notifications.error('Nome existe!')
            return
        s.add(EspacoCultural(nome=esp_nome.value, rua=esp_rua.value, numero=esp_num.value, bairro=esp_bairro.value))
        s.commit(); pn.state.notifications.success('Inserido!'); on_con_esp()
    finally: s.close()

def on_att_esp(e=None):
    if not esp_id.value: return
    s = Session()
    try:
        if o:=s.get(EspacoCultural, esp_id.value):
            changed = (o.nome != esp_nome.value or o.rua != esp_rua.value or o.numero != str(esp_num.value or o.bairro != esp_bairro.value))
            if not changed:
                pn.state.notifications.warning('Nenhuma alteração')
                return
            o.nome=esp_nome.value; o.bairro=esp_bairro.value; o.rua=esp_rua.value; o.bairro=esp_bairro.value; o.numero=esp_num.value; o.bairro=esp_bairro.value
            s.commit(); pn.state.notifications.success('Ok'); on_con_esp()
    finally: s.close()

def on_del_esp(e=None):
    s = Session()
    try: 
        if o:=s.get(EspacoCultural, esp_id.value): s.delete(o); s.commit(); pn.state.notifications.success('Del'); on_con_esp()
    finally: s.close()

btnConEsp.on_click(on_con_esp); btnLimparEsp.on_click(on_limpar_esp)
btnInsEsp.on_click(on_ins_esp); btnAttEsp.on_click(on_att_esp); btnDelEsp.on_click(on_del_esp)

on_con_esp()
tab_espacos = pn.Row(col_esp_main, tab_esp_admin, col_feed_esp_nav, name='Espaços')


In [ ]:
# --- 9. GESTÃO DE USUÁRIOS (Admin/Gerente) ---
user_id = pn.widgets.IntInput(name='ID', disabled=True, width=80)
user_nome = pn.widgets.TextInput(name='Nome')
user_email = pn.widgets.TextInput(name='Email')
user_cpf = pn.widgets.TextInput(name='CPF/RG')
user_pass = pn.widgets.PasswordInput(name='Senha (Deixe vazio para manter)')
user_role_select = pn.widgets.Select(name='Papel', options=['COMUM'])

btnConUser = pn.widgets.Button(name='Consultar', button_type='primary')
btnInsUser = pn.widgets.Button(name='Criar Usuário', button_type='success')
btnAttUser = pn.widgets.Button(name='Atualizar', button_type='warning')
btnDelUser = pn.widgets.Button(name='Excluir', button_type='danger')
btnLimparUser = pn.widgets.Button(name='Limpar', button_type='default')

tab_users = pn.widgets.Tabulator(pagination='remote', page_size=10, layout='fit_columns', selectable=1, show_index=False, disabled=True)

col_crud_user = pn.Column('### Gerenciar Usuários', user_id, user_nome, user_email, user_cpf, user_pass, user_role_select,
                          pn.Row(btnConUser, btnLimparUser, btnInsUser, btnAttUser, btnDelUser))

def on_con_users(e=None):
    if not CURRENT_USER: return
    try:
        my_role = str(CURRENT_USER.get('role', '')).upper()
        where_clause = ""
        if 'GERENTE' in my_role:
            where_clause = "WHERE r.nome = 'COMUM'"
        elif 'ADMIN' in my_role:
            where_clause = "WHERE r.nome IN ('GERENTE', 'COMUM')"
        
        query = f"""
            SELECT u.id_usuario, u.nome, u.email, u.cpf_rg, r.nome as role 
            FROM usuario u 
            JOIN role r ON u.id_papel = r.id_papel 
            {where_clause}
            ORDER BY u.id_usuario
        """
        tab_users.value = pd.read_sql_query(query, cnx)
    except Exception as ex: print(f'Erro users: {ex}')

def get_role_id_by_name(rname):
    s = Session(); r = s.query(Role).filter(Role.nome.ilike(f'%{rname}%')).first(); s.close()
    return r.id_papel if r else 2 

def on_sel_user(event):
    if not event.new: return
    row = tab_users.value.iloc[event.new[0]]
    user_id.value = int(row['id_usuario'])
    user_nome.value = str(row['nome'])
    user_email.value = str(row['email'])
    user_cpf.value = str(row['cpf_rg'])
    user_pass.value = ''
    user_role_select.value = str(row['role']).upper()
tab_users.param.watch(on_sel_user, 'selection')

def on_ins_user(e=None):
    s = Session()
    try:
        if s.query(Usuario).filter(or_(Usuario.email==user_email.value, Usuario.cpf_rg==user_cpf.value)).first():
            pn.state.notifications.error('Email ou CPF já existe')
            return
        rid = get_role_id_by_name(user_role_select.value)
        s.add(Usuario(nome=user_nome.value, email=user_email.value, cpf_rg=user_cpf.value, senha_hash=user_pass.value, id_papel=rid))
        s.commit(); pn.state.notifications.success('Usuário criado!'); on_con_users()
    except Exception as x: pn.state.notifications.error(f'Erro: {x}')
    finally: s.close()

def on_att_user(e=None):
    if not user_id.value: return
    s = Session()
    try:
        u = s.get(Usuario, user_id.value)
        if u:
            # DETECT CHANGES
            new_role_id = get_role_id_by_name(user_role_select.value)
            changed = (u.nome != user_nome.value or 
                       u.email != user_email.value or 
                       u.cpf_rg != user_cpf.value or 
                       u.id_papel != new_role_id or 
                       user_pass.value != '')
            
            if not changed:
                pn.state.notifications.warning('Nenhuma alteração detectada')
                return

            u.nome = user_nome.value
            u.email = user_email.value
            u.cpf_rg = user_cpf.value
            if user_pass.value: 
                u.senha_hash = user_pass.value
            u.id_papel = new_role_id
            s.commit(); pn.state.notifications.success('Usuário atualizado!'); on_con_users()
    except Exception as x: pn.state.notifications.error(f'Erro: {x}')
    finally: s.close()

def on_del_user(e=None):
    s = Session()
    try:
        if user_id.value == CURRENT_USER.get('id'):
            pn.state.notifications.error('Não pode se excluir!')
            return
        if o:=s.get(Usuario, user_id.value): s.delete(o); s.commit(); pn.state.notifications.success('Excluído'); on_con_users()
    finally: s.close()

def on_limpar_user(e=None):
    user_id.value = 0
    user_nome.value = ''
    user_email.value = ''
    user_cpf.value = ''
    user_pass.value = ''
    tab_users.selection = []
    on_con_users()

btnConUser.on_click(on_con_users)
btnInsUser.on_click(on_ins_user); btnDelUser.on_click(on_del_user)
btnAttUser.on_click(on_att_user); btnLimparUser.on_click(on_limpar_user)
on_con_users()
tab_usuarios = pn.Row(col_crud_user, tab_users, name='Usuários')


In [ ]:
# --- 10. ARTISTAS ---

# Widgets
art_id = pn.widgets.IntInput(name='ID', disabled=True, width=80)
art_nome = pn.widgets.TextInput(name='Nome')
art_cpf = pn.widgets.TextInput(name='CPF/RG')
art_email = pn.widgets.TextInput(name='Email')
art_tel = pn.widgets.TextInput(name='Telefone')
art_desc = pn.widgets.TextAreaInput(name='Descrição', height=100)

# Buttons
btnConArt = pn.widgets.Button(name='Consultar', button_type='primary')
btnLimparArt = pn.widgets.Button(name='Limpar', button_type='default')
btnInsArt = pn.widgets.Button(name='Inserir', button_type='success')
btnAttArt = pn.widgets.Button(name='Atualizar', button_type='warning')
btnDelArt = pn.widgets.Button(name='Excluir', button_type='danger')

# Layout
col_inputs_art = pn.Column('### Gestão de Artistas', 
                           art_id, art_nome, art_cpf, 
                           pn.Row(art_email, art_tel), 
                           art_desc,
                           pn.Row(btnConArt, btnLimparArt))

row_crud_art_btns = pn.Row(btnInsArt, btnAttArt, btnDelArt)
col_art_main = pn.Column(col_inputs_art, row_crud_art_btns)

# Table
tab_art_admin = pn.widgets.Tabulator(pagination='remote', page_size=10, layout='fit_columns', 
                                     selectable=1, show_index=False, disabled=True,
                                     sizing_mode='stretch_width', theme='bootstrap4')

# Logic
def carregar_form_art(row):
    try:
        art_id.value = int(row['id_artista'])
        art_nome.value = str(row['nome'])
        art_cpf.value = str(row['cpf_rg']) if row['cpf_rg'] else ''
        art_email.value = str(row['email']) if row['email'] else ''
        art_tel.value = str(row['telefone']) if row['telefone'] else ''
        art_desc.value = str(row['descricao']) if row['descricao'] else ''
    except: pass

def on_sel_art(event):
    if event.new:
         try:
             row = tab_art_admin.value.iloc[event.new[0]]
             carregar_form_art(row)
         except: pass
tab_art_admin.param.watch(on_sel_art, 'selection')

def fetch_artistas(limit=None, offset=0):
    n = art_nome.value.replace("'", "''")
    query = f"SELECT * FROM artista WHERE ('{n}'='' OR nome ILIKE '%%{n}%%') ORDER BY id_artista DESC"
    if limit:
        query += f" LIMIT {limit} OFFSET {offset} "
    return pd.read_sql_query(query, cnx)

def on_con_art(e=None):
    try:
        tab_art_admin.value = fetch_artistas()
    except Exception as x: pn.state.notifications.error(f'{x}')

def on_limpar_art(e=None):
    art_id.value = 0; art_nome.value = ''; art_cpf.value = ''
    art_email.value = ''; art_tel.value = ''; art_desc.value = ''
    tab_art_admin.selection = []
    on_con_art()

def on_ins_art(e=None):
    if not art_nome.value: return
    s = Session()
    try:
        # Check Duplicates CPF/Email
        if s.query(Artista).filter((Artista.cpf_rg==art_cpf.value) | (Artista.email==art_email.value)).first():
             pn.state.notifications.error('CPF ou Email duplicado!')
             return

        new_art = Artista(nome=art_nome.value, cpf_rg=art_cpf.value,
                          email=art_email.value, telefone=art_tel.value,
                          descricao=art_desc.value)
        s.add(new_art); s.commit(); pn.state.notifications.success('Inserido!'); on_con_art()
    except Exception as x: pn.state.notifications.error(f'{x}')
    finally: s.close()

def on_att_art(e=None):
    if not art_id.value: return
    s = Session()
    try:
        a = s.get(Artista, art_id.value)
        if a:
             a.nome = art_nome.value
             a.cpf_rg = art_cpf.value
             a.email = art_email.value
             a.telefone = art_tel.value
             a.descricao = art_desc.value
             s.commit(); pn.state.notifications.success('Atualizado!'); on_con_art()
    except Exception as x: pn.state.notifications.error(f'{x}')
    finally: s.close()

def on_del_art(e=None):
    s = Session()
    try:
        if a:=s.get(Artista, art_id.value): s.delete(a); s.commit(); pn.state.notifications.success('Deletado'); on_con_art()
    finally: s.close()

btnConArt.on_click(on_con_art); btnLimparArt.on_click(on_limpar_art)
btnInsArt.on_click(on_ins_art); btnAttArt.on_click(on_att_art); btnDelArt.on_click(on_del_art)

# Initial Load
# on_con_art()
tab_artistas = pn.Row(col_art_main, tab_art_admin, name='Artistas')


In [ ]:
# --- 10. APP PRINCIPAL ---
header_user = pn.pane.Markdown('')
btnHeaderLogin = pn.widgets.Button(name='Login', button_type='primary', width=100)
btnHeaderLogout = pn.widgets.Button(name='Logout', button_type='default', width=100, visible=False)

tabs_main = pn.Tabs(dynamic=True)

# Login View
log_email = pn.widgets.TextInput(name='Email')
log_pass = pn.widgets.PasswordInput(name='Senha')
btnLog = pn.widgets.Button(name='Entrar', button_type='primary')
btnCancelLog = pn.widgets.Button(name='Cancelar')
log_alert = pn.pane.Alert('', visible=False, alert_type='danger')

def do_login(e):
    if tentar_login(log_email.value, log_pass.value):
        log_alert.visible = False
        atualizar_interface()
        on_con_users() # Trigger User Refresh
        root.clear(); root.append(app_view)
        pn.state.notifications.success(f'Bem-vindo {CURRENT_USER["nome"]}')
    else:
        log_alert.object = 'Falha login'; log_alert.visible = True

btnLog.on_click(do_login)
btnCancelLog.on_click(lambda e: (root.clear(), root.append(app_view)))
login_card = pn.Card(log_email, log_pass, pn.Row(btnLog, btnCancelLog), log_alert, title='Acesso ao Sistema', width=400)
login_view = pn.Column(login_card, align='center')

def go_login(e): root.clear(); root.append(login_view)
def do_logout(e): logout(); atualizar_interface()

btnHeaderLogin.on_click(go_login)
btnHeaderLogout.on_click(do_logout)

app_view = pn.Column(
    pn.Row('### Gestão Cultural', pn.Spacer(), header_user, btnHeaderLogin, btnHeaderLogout),
    tabs_main
)

# Inicializa como Visitante
atualizar_interface()
root = pn.Column(app_view)
root.servable()
